# 2 Data Wrangling <a id='2_Data_wrangling'></a>

## Contents<a id='2.1_Contents'></a>

### Part A <a id='Part_A'></a>
* [2.1 Introduction](#2.1_Introduction)
  * [2.1.1 Recap Of Data Science Problem](#2.1.1_Recap_Of_Data_Science_Problem)
* [2.2 Imports](#2.2_Imports)
* [2.3 Files](#2.3_Files)
* [2.4 Columns](#2.4_Columns)
* [2.5 Objectives](#2.5_Objectives)
* [2.6 Load the Data](#2.6_Load_the_Data)
* [2.7 Explore the Data](#2.7_Explore_the_Data)
    * [2.7.1 Admissions Data](#2.7.1_Admissions_Data)       
    * [2.7.2 Disease Data](#2.7.2_Disease_Data)    
    * [2.7.3 Labs Data](#2.7.3_Labs_Data)
    * [2.7.4 Patients Data](#2.7.4_Patients_Data)
    * [2.7.5 Prescriptions Data](#2.7.5_Prescriptions_Data)    
    * [2.7.6 Train Data](#2.7.6_Train_Data)

### Part B <a id='Part_B'></a> 
* [2.8 Subset the Data](#2.8_Subset_the_Data)
    * [2.8.1 Subset the Disease Data](#2.8.1_Subset_Admissions_Data)       
    * [2.8.2 Subset the Admissions Data](#2.8.2_Subset_Admission_Data)    
    * [2.8.3 Subset the Labs Data](#2.8.3_Subset_Labs_Data)    
    * [2.8.4 Subset the Prescriptions Data](#2.8.4_Subset_Prescriptions_Data)    
* [2.9 One Record per Member Id](#2.9_One_Record_per_Member_Id)
    * [2.9.1  Admissions Data Converted](#2.9.1_Admissions_Data) 
    * [2.9.2  Disease Data Converted](#2.9.1_Disease_Data)    
    * [2.9.3  Labs Data Converted](#2.9.3_Labs_Data)   
    * [2.9.4 Patients Data Converted](#2.9.4_Patients_Data)    
    * [2.9.5  Prescriptions Data Converted](#2.9.5_Prescriptions_Data)
   
### Part C <a id='Part_C'></a>  
* [2.10  Join the Converted Dataframes](#2.10_Join_the_Dataframes)
* [2.11  Save the Final Dataframe](#2.11_Save_the_Data)
* [2.12  Final Remarks](#2.12_Final_Remarks)
     

## 2.1 Introduction<a id='2.1_Introduction'></a>

This step focuses on collecting the data, organizing, and making sure factors are well defined. The majority of the data will be cleaned in this step.

### 2.1.1 Recap Of Data Science Problem<a id='2.1.1_Recap_Of_Data_Science_Problem'></a>

HCC is contingent on ICD-10 coding to attribute risk scores to patients. Along with demographic factors (such as age and gender), Anthem uses HCC coding to assign patients a RAF. HCC coding provides Anthem with a method of evaluating health risk(s) in order to predict the costs associated with each patient. The purpose of this data science project is to come up with a model for predicting new diseases for 2020 using data from 2018-2019 period. 

## 2.2 Imports<a id='2.2_Imports'></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
#from library.sb_utils import save_file

from ipynb.fs.full.auxiliary_functions import save_file

from ipynb.fs.full.auxiliary_functions import extract_icd_code 
# Extracts the icd codes from the icd_code column and return a dictionary where keys are member_ids and values are the corresponding icd codes, 
# see 'auxiliary_functions.ipynb' in the directory.

from ipynb.fs.full.auxiliary_functions import df1_one_record_per_member_id 
# Creates a df out of df1_subset where there is one record per member id

from ipynb.fs.full.auxiliary_functions import fill_df1_res 
# Fills df1_res

from ipynb.fs.full.auxiliary_functions import df2_one_record_per_member_id 
# Creates a df out of df2_subset where there is one record per member id

from ipynb.fs.full.auxiliary_functions import fill_df2_res 
# Fills the df2_res 

from ipynb.fs.full.auxiliary_functions import df3_one_record_per_member_id 
# Creates a df out of df3_subset where there is one record per member id

from ipynb.fs.full.auxiliary_functions import fill_df3_res 
# Fills df3_res

from ipynb.fs.full.auxiliary_functions import df5_one_record_per_member_id 
# Creates a df out of df5_subset where there is one record per member id

from ipynb.fs.full.auxiliary_functions import fill_df5_res 
# Fills df5_res

## 2.3 Files<a id='2.3_Files'></a> 


* admissions_data.csv - contains information about hospital admissions for 2018-2019
* disease_data.csv - contains information about ICD and HCC for 2018-2019
* labs_data.csv - contains information about laboratory tests for 2018-2019
* patients_data.csv - contains member data such as gender, age, etc. for 2018-2019
* prescriptions_data.csv - contains drug prescriptions for 2018-2019
* train_labels.csv - contains labels for 2019 (new diseases discovered that year)


## 2.4 Columns<a id='2.4_Columns'></a>

* member_id - patients' ID
* admission_date - a date when hospital admission occurred
* discharge_date - date when member was discharge from the hospital
* admission_type - a hospital admission type
* icd_code - an ICD code (diagnosis) used for hospital admission
* is_readmission - 1 if member was already in the hospital in last 30 days, otherwise 0
* er_to_inp_admission_transfer - 1 if member had inpatient (INP) and ER (emergency) hospital admission same day, otherwise 0
* days_to_prev_admission - a number of days since last admission
* year_of_service - a year when ICD / HCC was detected by doctors
* icd_code - an ICD code (disease)
* icd_chronic_or_acute - a chronic / acute disease type
* hcc_code - a HCC coding for ICD
* hcc_chronic_or_acute - a chronic / acute HCC type
* date_of_service - a date when some service was provided to patients
* loinc_code - a laboratory test code
* performed_test_name - a laboratory test name
* abnormal_code - a flag that tells about low (L) or high (H) lab results
* result_value - a result value
* normal_low_value_numeric - a lower bound for reference lab result
* normal_high_value_numeric - an upper bound for reference lab result
* date_of_service - a date when some service was provided to patients
* patient_age - a patient's age
* patient_gender - a patient's gender
* patient_age - a patient's age
* dual_status - a flag that tells about Medicaid / Medicate insurance programms
* patient_age - a patient's age
* insurance_company - a company providing medical insurance
* insurance_type - an insurance type
* pbp_type - a member's risk group
* county - a member's county
* city - a member's city
* date_filled - a date when prescription was filled for the member
* ndc_number - a unique drug ID
* days_supply - a number of days for which the prescription was filled
* drug_name - a drug name
* metric_quantity - a drug dosage
* new_or_refill - 1 if a new drug was prescribed, otherwise 0
* hcc_9 - a flag that identifies that member has HCC 9. Similar logic should be used for other hcc_N columns

## 2.5 Objectives<a id='2.5_Objectives'></a>

There are some fundamental questions to resolve in this notebook before we move on:

   * What are the required target ilnesses?
   * Are there potentially useful features?
   * Are there any fundamental issues with the data?

## 2.6 Load the Data<a id='2.6_Load_the_Data'></a>

In [2]:
df1 = pd.read_csv('admissions_data.csv')
df2 = pd.read_csv('disease_data.csv')
df3 = pd.read_csv('labs_data.csv')
df4 = pd.read_csv('patients_data.csv')
df5 = pd.read_csv('prescriptions_data.csv')
df6 = pd.read_csv('train_labels.csv')

## 2.7  Explore the Data<a id='2.7_Explore_the_Data'></a>

### 2.7.1 Admissions Data<a id='2.7.1_Admissions_Data'></a>

We call the info method to see a summary of the data. We also display the first few records using the head method.

In [3]:
df1.head()

,member_id,admission_date,discharge_date,admission_type,icd_code,is_readmission,er_to_inp_admission_transfer,days_to_prev_admission
0,AAoq6W9QR0G/kOfNZnoCPg==,2018-04-03,2018-04-03,OTH,NaN,No,N,NaN
1,AAoq6W9QR0G/kOfNZnoCPg==,2018-06-04,2018-06-04,OTH,NaN,No,N,62.0
2,AAoq6W9QR0G/kOfNZnoCPg==,2018-06-18,2018-06-18,OTH,NaN,No,N,14.0
3,AAoq6W9QR0G/kOfNZnoCPg==,2018-10-11,2018-10-11,OTH,NaN,No,N,115.0
4,AAoq6W9QR0G/kOfNZnoCPg==,2018-11-02,2018-11-02,OTH,NaN,No,N,22.0


In [4]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26479 entries, 0 to 26478
Data columns (total 8 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   member_id                     26479 non-null  object 
 1   admission_date                26479 non-null  object 
 2   discharge_date                26479 non-null  object 
 3   admission_type                26479 non-null  object 
 4   icd_code                      18792 non-null  object 
 5   is_readmission                26479 non-null  object 
 6   er_to_inp_admission_transfer  26479 non-null  object 
 7   days_to_prev_admission        11508 non-null  float64
dtypes: float64(1), object(7)
memory usage: 1.6+ MB


In [5]:
df1.nunique()

member_id                       9354
admission_date                   730
discharge_date                   755
admission_type                     9
icd_code                        5829
is_readmission                     2
er_to_inp_admission_transfer       2
days_to_prev_admission           852
dtype: int64

In order to use the date time specific functions in Pandas, we need to convert `admission_date` and `discharge_date` into an appropriate format.  

In [6]:
df1['admission_date']=pd.to_datetime(df1['admission_date'])
df1['discharge_date']=pd.to_datetime(df1['discharge_date'])
df1.info() # we double check the data type

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26479 entries, 0 to 26478
Data columns (total 8 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   member_id                     26479 non-null  object        
 1   admission_date                26479 non-null  datetime64[ns]
 2   discharge_date                26479 non-null  datetime64[ns]
 3   admission_type                26479 non-null  object        
 4   icd_code                      18792 non-null  object        
 5   is_readmission                26479 non-null  object        
 6   er_to_inp_admission_transfer  26479 non-null  object        
 7   days_to_prev_admission        11508 non-null  float64       
dtypes: datetime64[ns](2), float64(1), object(5)
memory usage: 1.6+ MB


In our analysis we only consider records from 2018 and 2019. We will check unique values for the years of `admission_date` and `discharge_date` columns to see whether we need to eliminate any records that do not meet that criteria.

In [7]:
pd.DatetimeIndex(df1['admission_date']).year.unique()

Int64Index([2018, 2019], dtype='int64', name='admission_date')

In [8]:
pd.DatetimeIndex(df1['discharge_date']).year.unique()

Int64Index([2018, 2019, 2020], dtype='int64', name='discharge_date')

In [9]:
df1=df1[df1['discharge_date']<='2019-12-31']           # eliminate records from 2020
pd.DatetimeIndex(df1['discharge_date']).year.unique()  # check the update

Int64Index([2018, 2019], dtype='int64', name='discharge_date')

In [10]:
df1[(df1['discharge_date']-df1['admission_date']).dt.days<0].head(3)

,member_id,admission_date,discharge_date,admission_type,icd_code,is_readmission,er_to_inp_admission_transfer,days_to_prev_admission
475,BCe7vxpdQd+7exQf6EjkMQ==,2019-07-02,2019-06-20,INP,M1611,No,N,12.0
3576,IvBP0/7WQ/+IA0oLycHUDw==,2018-07-10,2018-06-21,INP,M1712;M25562,No,N,19.0
5310,MtKBVgQaQQWMV7/u5A6L7Q==,2019-12-02,2019-11-07,INP,M48061;M5136,Yes,N,15.0


Now `discharge_date` must be later than `admission_date` for each patient. However, when we checked the difference of these columns (see above) we saw that we have 13 records with negative difference, which must be an error. This error might be that the dates were swapped when entered. To fix this issue we will swap the dates for these records (see below).

In [11]:
num_days = (df1['discharge_date']-df1['admission_date']).dt.days<0 # durations that are smaller than zero.

df1.loc[num_days,['admission_date','discharge_date']] = df1.loc[num_days,['discharge_date','admission_date']].values.astype('datetime64[D]')

In [12]:
df1[(df1['discharge_date']-df1['admission_date']).dt.days<0] # check the update

,member_id,admission_date,discharge_date,admission_type,icd_code,is_readmission,er_to_inp_admission_transfer,days_to_prev_admission


Next we will add one more column to keep track of the duration of each visit.

In [13]:
df1['visit_duration']=(df1['discharge_date']-df1['admission_date']).dt.days
df1.head(3) # check the update

,member_id,admission_date,discharge_date,admission_type,icd_code,is_readmission,er_to_inp_admission_transfer,days_to_prev_admission,visit_duration
0,AAoq6W9QR0G/kOfNZnoCPg==,2018-04-03 00:00:00,2018-04-03 00:00:00,OTH,NaN,No,N,NaN,0
1,AAoq6W9QR0G/kOfNZnoCPg==,2018-06-04 00:00:00,2018-06-04 00:00:00,OTH,NaN,No,N,62.0,0
2,AAoq6W9QR0G/kOfNZnoCPg==,2018-06-18 00:00:00,2018-06-18 00:00:00,OTH,NaN,No,N,14.0,0


Notice that if a patient was admitted and discharged on the same day then the duration of the visit is 0. To distinguish a patient with a visit of 0 duration from the patients who did not visit the hospital we will create a column whose values are equal to `visit_duration` values increased by 1.   

In [14]:
df1['visit_duration_updated']=df1['visit_duration']+1
df1.head(10) # check the update

,member_id,admission_date,discharge_date,admission_type,icd_code,is_readmission,er_to_inp_admission_transfer,days_to_prev_admission,visit_duration,visit_duration_updated
0,AAoq6W9QR0G/kOfNZnoCPg==,2018-04-03 00:00:00,2018-04-03 00:00:00,OTH,NaN,No,N,NaN,0,1
1,AAoq6W9QR0G/kOfNZnoCPg==,2018-06-04 00:00:00,2018-06-04 00:00:00,OTH,NaN,No,N,62.0,0,1
2,AAoq6W9QR0G/kOfNZnoCPg==,2018-06-18 00:00:00,2018-06-18 00:00:00,OTH,NaN,No,N,14.0,0,1
3,AAoq6W9QR0G/kOfNZnoCPg==,2018-10-11 00:00:00,2018-10-11 00:00:00,OTH,NaN,No,N,115.0,0,1
4,AAoq6W9QR0G/kOfNZnoCPg==,2018-11-02 00:00:00,2018-11-02 00:00:00,OTH,NaN,No,N,22.0,0,1
5,AAoq6W9QR0G/kOfNZnoCPg==,2018-11-27 00:00:00,2018-11-27 00:00:00,OTH,NaN,No,N,25.0,0,1
6,ABVROrC0SFCeFKI7k2dCLg==,2018-03-30 00:00:00,2018-03-30 00:00:00,INP,R55,No,N,NaN,0,1
7,ABounsImR7yi6ARWw/pLkQ==,2019-12-30 00:00:00,2019-12-30 00:00:00,ER,NaN,No,N,NaN,0,1
8,ACKVU9XTTqKWeNs8SZmUkg==,2019-03-28 00:00:00,2019-03-28 00:00:00,ER,NaN,No,N,NaN,0,1
9,ACKVU9XTTqKWeNs8SZmUkg==,2019-04-03 00:00:00,2019-04-08 00:00:00,ER,K8020;I4891,Yes,Y,6.0,5,6


In [15]:
df1['visit_duration'].unique() # check the update

array([  0,   5,   2,  15,  10,  28,   1,   3,  23,  42,   8,   6,  11,
         9,   4,  19,  18,   7,  27,  12,  16,  20,  13,  31,  88,  52,
        17,  36,  21,  14,  22,  38,  24,  26,  29,  30,  41,  33,  32,
        25,  39,  35,  51,  44,  37,  43,  40,  53,  57,  46, 114,  73,
        34,  66,  64,  54,  92,  63,  69,  45,  83,  50,  56,  55,  62,
        48,  47, 142, 160, 116,  71,  81, 122, 121,  49,  59,  78,  58,
        70,  65,  67, 156, 108, 303, 113])

Furthermore, `icd_code` column has 3 main types of values: null, a single icd code, or two or more icd codes joint with a (or multiple) semicolon(s) (see below). 

In [16]:
df1['icd_code'].unique()

array([nan, 'R55', 'K8020;I4891', ..., 'K810;Z833', 'G459;I509',
       'R0602;J9600'], dtype=object)

Using `icd_code` column of df1 we will make a dictionary that has member ids as its keys, and a list containing all the icd codes assigned to the member id. We will use a function called  'extract_icd_code' for this task (see below). The dictionary will be saved for future use. 

In [17]:
df1_icd_codes=extract_icd_code(df1)

In [18]:
# df1_icd_codes is a very large dictionary so we will not show it here.

In [19]:
# to see how an item in df1_icd_codes dictionary looks like
df1_icd_codes['AEeiIrq1RKOuuHYqsgeVJg==']

['I6522', 'J449', 'R0602', 'G8918', 'I509']

We observe that most of values for `days_to_prev_admission` is null.

In [20]:
df1['days_to_prev_admission'].value_counts(dropna=False)

NaN       14903
0.0         677
5.0         203
4.0         195
6.0         194
          ...  
888.0         1
1004.0        1
860.0         1
988.0         1
1031.0        1
Name: days_to_prev_admission, Length: 849, dtype: int64

In [21]:
df1['admission_date']=pd.to_datetime(df1['admission_date'])
df1['nth_visit']=df1.groupby('member_id')['admission_date'].rank(method='first')
df1['nth_visit']=df1['nth_visit'].astype(int)
df1.head(3)

,member_id,admission_date,discharge_date,admission_type,icd_code,is_readmission,er_to_inp_admission_transfer,days_to_prev_admission,visit_duration,visit_duration_updated,nth_visit
0,AAoq6W9QR0G/kOfNZnoCPg==,2018-04-03,2018-04-03 00:00:00,OTH,NaN,No,N,NaN,0,1,1
1,AAoq6W9QR0G/kOfNZnoCPg==,2018-06-04,2018-06-04 00:00:00,OTH,NaN,No,N,62.0,0,1,2
2,AAoq6W9QR0G/kOfNZnoCPg==,2018-06-18,2018-06-18 00:00:00,OTH,NaN,No,N,14.0,0,1,3


Next we will drop the columns that we will not consider for our model. 

In [22]:
df1.drop(['admission_date', 'discharge_date', 'icd_code'], axis=1, inplace=True)
df1.head()

,member_id,admission_type,is_readmission,er_to_inp_admission_transfer,days_to_prev_admission,visit_duration,visit_duration_updated,nth_visit
0,AAoq6W9QR0G/kOfNZnoCPg==,OTH,No,N,NaN,0,1,1
1,AAoq6W9QR0G/kOfNZnoCPg==,OTH,No,N,62.0,0,1,2
2,AAoq6W9QR0G/kOfNZnoCPg==,OTH,No,N,14.0,0,1,3
3,AAoq6W9QR0G/kOfNZnoCPg==,OTH,No,N,115.0,0,1,4
4,AAoq6W9QR0G/kOfNZnoCPg==,OTH,No,N,22.0,0,1,5


In [23]:
df1_duplicates=df1[df1.duplicated()]
df1_duplicates # check duplicates

,member_id,admission_type,is_readmission,er_to_inp_admission_transfer,days_to_prev_admission,visit_duration,visit_duration_updated,nth_visit


### 2.7.2 Disease Data<a id='2.7.2_Disease_Data'></a>

In [24]:
df2.head()

,member_id,year_of_service,icd_code,icd_chronic_or_acute,hcc_code,hcc_chronic_or_acute
0,AAHLhKW7RYOEQIemWrh/1w==,2018,I739,Chronic,108.0,Chronic
1,AAHLhKW7RYOEQIemWrh/1w==,2018,M545,Unknown,NaN,NaN
2,AAHLhKW7RYOEQIemWrh/1w==,2019,N183,Chronic,138.0,Chronic
3,AAHLhKW7RYOEQIemWrh/1w==,2019,E6601,Chronic,22.0,Chronic
4,AAHLhKW7RYOEQIemWrh/1w==,2019,J8410,Chronic,112.0,Chronic


In [25]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1022161 entries, 0 to 1022160
Data columns (total 6 columns):
 #   Column                Non-Null Count    Dtype  
---  ------                --------------    -----  
 0   member_id             1022161 non-null  object 
 1   year_of_service       1022161 non-null  int64  
 2   icd_code              1022154 non-null  object 
 3   icd_chronic_or_acute  1022161 non-null  object 
 4   hcc_code              320286 non-null   float64
 5   hcc_chronic_or_acute  320286 non-null   object 
dtypes: float64(1), int64(1), object(4)
memory usage: 46.8+ MB


In [26]:
df2.nunique()

member_id               37071
year_of_service             3
icd_code                12959
icd_chronic_or_acute        3
hcc_code                   68
hcc_chronic_or_acute        2
dtype: int64

We have some records from 2021 besides 2018 and 2019. Since our analysis focuses on 2018 and 2019 only, we will ignore the records with the year 2021.

In [27]:
df2['year_of_service'].unique()

array([2018, 2019, 2021])

In [28]:
df2=df2[df2['year_of_service']< 2021] # eliminate records from 2021
df2['year_of_service'].unique()

array([2018, 2019])

Also, we will drop `year_of_service` column all together since we will not consider the date and time in our model.

In [29]:
df2.drop(['year_of_service'], axis=1, inplace=True)
df2.head() # check the update

,member_id,icd_code,icd_chronic_or_acute,hcc_code,hcc_chronic_or_acute
0,AAHLhKW7RYOEQIemWrh/1w==,I739,Chronic,108.0,Chronic
1,AAHLhKW7RYOEQIemWrh/1w==,M545,Unknown,NaN,NaN
2,AAHLhKW7RYOEQIemWrh/1w==,N183,Chronic,138.0,Chronic
3,AAHLhKW7RYOEQIemWrh/1w==,E6601,Chronic,22.0,Chronic
4,AAHLhKW7RYOEQIemWrh/1w==,J8410,Chronic,112.0,Chronic


We observed that more than 68% of `icd_chronic_or_acute` is null (entered as 'Unknown') and the rest of the values are chronic except 2 of them (see below). The column `hcc_chronic_or_acute` is NaN whenever `icd_chronic_or_acute` is 'Unknown'. Also we have 2584 (approximately 0.002 % of the total) records where `icd_chronic_or_acute` is 'Chronic' but `hcc_chronic_or_acute` is 'Acute'. We will not make any changes on these records since there are only a few of them. Consequently, we will eliminate `hcc_chronic_or_acute` column entirely and include only `icd_chronic_or_acute` based on these findings. 

In [30]:
df2['icd_chronic_or_acute'].value_counts()

Unknown    698326
Chronic    320828
Acute           2
Name: icd_chronic_or_acute, dtype: int64

In [31]:
df2[df2['icd_chronic_or_acute']=='Unknown']['hcc_chronic_or_acute'].value_counts()

Series([], Name: hcc_chronic_or_acute, dtype: int64)

In [32]:
df2[df2['icd_chronic_or_acute']=='Chronic']['hcc_chronic_or_acute'].value_counts()

Chronic    317700
Acute        2584
Name: hcc_chronic_or_acute, dtype: int64

In [33]:
df2[df2['icd_chronic_or_acute']=='Acute']['hcc_chronic_or_acute'].value_counts()

Chronic    2
Name: hcc_chronic_or_acute, dtype: int64

The distribution of the values of `hcc_chronic_or_acute` is as follows:

In [34]:
df2['icd_chronic_or_acute'].value_counts()

Unknown    698326
Chronic    320828
Acute           2
Name: icd_chronic_or_acute, dtype: int64

In [35]:
df2.drop(['hcc_chronic_or_acute'], axis=1, inplace=True)
df2.head(3) # check the update

,member_id,icd_code,icd_chronic_or_acute,hcc_code
0,AAHLhKW7RYOEQIemWrh/1w==,I739,Chronic,108.0
1,AAHLhKW7RYOEQIemWrh/1w==,M545,Unknown,NaN
2,AAHLhKW7RYOEQIemWrh/1w==,N183,Chronic,138.0


Our ultimate goal is to predict hcc codes assigned to each patient. However, hcc codes are determined based on icd codes. We will drop `hcc_code` column to avoid any potential issues caused by the correlation. Once we determine the icd codes we will retrieve hcc codes using the icd codes.

In [36]:
df2.drop(['hcc_code'], axis=1, inplace=True)
df2.head() # check the update 

,member_id,icd_code,icd_chronic_or_acute
0,AAHLhKW7RYOEQIemWrh/1w==,I739,Chronic
1,AAHLhKW7RYOEQIemWrh/1w==,M545,Unknown
2,AAHLhKW7RYOEQIemWrh/1w==,N183,Chronic
3,AAHLhKW7RYOEQIemWrh/1w==,E6601,Chronic
4,AAHLhKW7RYOEQIemWrh/1w==,J8410,Chronic


In [37]:
# save the data to a new csv file
datapath = '../Outputs'
save_file(df2, 'df2_final.csv', datapath)

A file already exists with this name.

Do you want to overwrite? (Y/N)Y
Writing file.  "../Outputs/df2_final.csv"


In [38]:
df2_duplicates=df2[df2.duplicated()]
df2_duplicates.head() # ??? these are not duplicates?? check the duplicates

,member_id,icd_code,icd_chronic_or_acute
5,AAHLhKW7RYOEQIemWrh/1w==,I739,Chronic
61,AAKW/PKhRva+ZS2B4s0AmA==,M792,Unknown
64,AAKW/PKhRva+ZS2B4s0AmA==,R5383,Unknown
66,AAKW/PKhRva+ZS2B4s0AmA==,I25810,Unknown
67,AAKW/PKhRva+ZS2B4s0AmA==,E1142,Chronic


In [39]:
df2[df2['member_id']=='AAHLhKW7RYOEQIemWrh/1w==']

,member_id,icd_code,icd_chronic_or_acute
0,AAHLhKW7RYOEQIemWrh/1w==,I739,Chronic
1,AAHLhKW7RYOEQIemWrh/1w==,M545,Unknown
2,AAHLhKW7RYOEQIemWrh/1w==,N183,Chronic
3,AAHLhKW7RYOEQIemWrh/1w==,E6601,Chronic
4,AAHLhKW7RYOEQIemWrh/1w==,J8410,Chronic
5,AAHLhKW7RYOEQIemWrh/1w==,I739,Chronic
6,AAHLhKW7RYOEQIemWrh/1w==,L4050,Chronic


### 2.7.3 Labs Data<a id='2.7.3_Labs_Data'></a>

In [40]:
df3.head()

,member_id,date_of_service,loinc_code,performed_test_name,abnormal_code,result_value,normal_low_value_numeric,normal_high_value_numeric
0,OEQjkMBoTqK7YmVc02NQDQ==,2018-10-25,13457-7,LIPID PANEL WITH REFLEX TO DIRECT LDL,NaN,85000.0,0.0,99.0
1,OEQjkMBoTqK7YmVc02NQDQ==,2018-10-25,17861-6,CALCIUM,NaN,9600.0,8.6,10.4
2,OEQjkMBoTqK7YmVc02NQDQ==,2018-10-25,33914-3,EGFR NON-AFR. AMERICAN,NaN,91000.0,60.0,0.0
3,OEQjkMBoTqK7YmVc02NQDQ==,2018-10-25,4544-3,HEMATOCRIT,NaN,42800.0,35.0,45.0
4,OEQjkMBoTqK7YmVc02NQDQ==,2018-10-25,713-8,EOSINOPHILS,NaN,6800.0,0.0,0.0


In [41]:
df3.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5918612 entries, 0 to 5918611
Data columns (total 8 columns):
 #   Column                     Non-Null Count    Dtype  
---  ------                     --------------    -----  
 0   member_id                  5918612 non-null  object 
 1   date_of_service            5918612 non-null  object 
 2   loinc_code                 5918612 non-null  object 
 3   performed_test_name        5915984 non-null  object 
 4   abnormal_code              993458 non-null   object 
 5   result_value               4977832 non-null  float64
 6   normal_low_value_numeric   5123901 non-null  float64
 7   normal_high_value_numeric  5257167 non-null  float64
dtypes: float64(3), object(5)
memory usage: 361.2+ MB


In [42]:
df3.nunique()

member_id                    24483
date_of_service                706
loinc_code                    3078
performed_test_name           5895
abnormal_code                   26
result_value                 15861
normal_low_value_numeric       365
normal_high_value_numeric      613
dtype: int64

We will convert the data type of `date_of_service` into an appropriate format for additional analysis.

In [43]:
df3['date_of_service']=pd.to_datetime(df3['date_of_service'])

In [44]:
df3.info(verbose=True, null_counts=True) #check the update

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5918612 entries, 0 to 5918611
Data columns (total 8 columns):
 #   Column                     Non-Null Count    Dtype         
---  ------                     --------------    -----         
 0   member_id                  5918612 non-null  object        
 1   date_of_service            5918612 non-null  datetime64[ns]
 2   loinc_code                 5918612 non-null  object        
 3   performed_test_name        5915984 non-null  object        
 4   abnormal_code              993458 non-null   object        
 5   result_value               4977832 non-null  float64       
 6   normal_low_value_numeric   5123901 non-null  float64       
 7   normal_high_value_numeric  5257167 non-null  float64       
dtypes: datetime64[ns](1), float64(3), object(4)
memory usage: 361.2+ MB


Below we checked years of services. As hoped for, we only have records from 2018 and 2019.  

In [45]:
pd.DatetimeIndex(df3['date_of_service']).year.unique()

Int64Index([2018, 2019], dtype='int64', name='date_of_service')

`loinc_code` column contains codes for the performed laboratory tests. `performed_test_name`  column contains the names of the performed laboratory tests. We will drop `performed_test_name` column since `loinc_code` is enough to encode the performed lab tests. 

In [46]:
df3.drop(['performed_test_name'], axis=1, inplace=True)
df3.head(3) # check the update

,member_id,date_of_service,loinc_code,abnormal_code,result_value,normal_low_value_numeric,normal_high_value_numeric
0,OEQjkMBoTqK7YmVc02NQDQ==,2018-10-25,13457-7,NaN,85000.0,0.0,99.0
1,OEQjkMBoTqK7YmVc02NQDQ==,2018-10-25,17861-6,NaN,9600.0,8.6,10.4
2,OEQjkMBoTqK7YmVc02NQDQ==,2018-10-25,33914-3,NaN,91000.0,60.0,0.0


The result values are significantly bigger than both normal low and normal high values. The following cell is justifying that this is a trend overall; the average of `result_value` column is 1,312,4710. However, the averages of `normal_low_value_numeric` and `normal_high_value_numeric` are ~54.5 and ~213.7, respectively. 

* After some careful examination we saw that each result value multiplied by 1000 before entered, which caused the issue. We divided each result value by 1000 to correct this abnormality. 


In [47]:
df3.describe()

,result_value,normal_low_value_numeric,normal_high_value_numeric
count,4.977832e+06,5.123901e+06,5.257167e+06
mean,1.312471e+07,5.455767e+01,2.137437e+02
std,2.868435e+10,3.353208e+02,1.960277e+03
min,-2.200000e+03,0.000000e+00,0.000000e+00
25%,4.500000e+03,0.000000e+00,1.035000e+00
50%,1.970000e+04,6.000000e+00,1.080000e+01
75%,8.300000e+04,3.200000e+01,9.900000e+01
max,6.399757e+13,2.500000e+04,9.999990e+05


In [48]:
df3['result_value']=df3['result_value']/1000.0
df3.head(3) # check the update

,member_id,date_of_service,loinc_code,abnormal_code,result_value,normal_low_value_numeric,normal_high_value_numeric
0,OEQjkMBoTqK7YmVc02NQDQ==,2018-10-25,13457-7,NaN,85.0,0.0,99.0
1,OEQjkMBoTqK7YmVc02NQDQ==,2018-10-25,17861-6,NaN,9.6,8.6,10.4
2,OEQjkMBoTqK7YmVc02NQDQ==,2018-10-25,33914-3,NaN,91.0,60.0,0.0


We examined the negative values entered for `result_value`, see the next cell.

In [175]:
df3[df3['result_value']<0].head(3)

KeyError: 'result_value'

In [50]:
df3[df3['result_value']<0].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16 entries, 1770719 to 5903684
Data columns (total 7 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   member_id                  16 non-null     object        
 1   date_of_service            16 non-null     datetime64[ns]
 2   loinc_code                 16 non-null     object        
 3   abnormal_code              3 non-null      object        
 4   result_value               16 non-null     float64       
 5   normal_low_value_numeric   13 non-null     float64       
 6   normal_high_value_numeric  15 non-null     float64       
dtypes: datetime64[ns](1), float64(3), object(3)
memory usage: 1.0+ KB


In [51]:
df3[df3['result_value']<0]['loinc_code'].value_counts()

73561-3    15
64084-7     1
Name: loinc_code, dtype: int64

The previous two cells show that we have 16 records where result value is a negative number. In all these records normal low and high values are not provided. Most of these records (15 out of 16) are lab test '73561-3', which has [-2,2] as its normal range of values. The remaining record belongs to lab test '64084-7', which must be higher than 0.5 to be considered as normal. We will fill the values accordingly, see the next cell. 

In [52]:
df3.loc[df3['loinc_code']=='73561-3','normal_low_value_numeric']=-2.0
df3.loc[df3['loinc_code']=='73561-3','normal_high_value_numeric']=2.0 # whether the result value is negative or not
df3.loc[df3['loinc_code']=='64084-7','normal_low_value_numeric']=0.5
df3[(df3['loinc_code']=='73561-3') | (df3['loinc_code']=='64084-7')].head() # check the update

,member_id,date_of_service,loinc_code,abnormal_code,result_value,normal_low_value_numeric,normal_high_value_numeric
17342,ORbIz1E4TFemyCvm7xl++w==,2019-10-28,64084-7,NaN,0.00,0.5,0.0
39165,OhpzMiWgReGqqZnFhq+cdg==,2018-12-27,64084-7,NaN,0.00,0.5,0.0
39171,OhpzMiWgReGqqZnFhq+cdg==,2018-12-27,64084-7,NaN,0.01,0.5,0.0
47910,OoBZBWT+QmCare0g+wFYyw==,2018-07-19,64084-7,NaN,0.38,0.5,0.0
71885,O55M4HBYSRCh2jilMoUyWQ==,2019-12-19,64084-7,,0.04,0.5,NaN


In [53]:
df3['abnormal_code'].unique()

array([nan, 'H', 'L', '     ', 'H    ', 'L    ', 'A', 'A    ', '<', '-',
       'HH', '>', 'LL', '<    ', 'Y', '>    ', '+', 'N    ', 'Y    ',
       'NORMA', 'HIGH ', 'BELOW', 'ABOVE', 'N', '+    ', '*    ', '-    '],
      dtype=object)

Now notice that `abnormal_code` has values that are not consistent (see the previous cell). We will update the values to 'H' and 'L' accordingly. We will also keep a copy of the column with the original values. 

In [54]:
df3['abnormal_code_updated']=df3['abnormal_code']
df3.head() # check the update

,member_id,date_of_service,loinc_code,abnormal_code,result_value,normal_low_value_numeric,normal_high_value_numeric,abnormal_code_updated
0,OEQjkMBoTqK7YmVc02NQDQ==,2018-10-25,13457-7,NaN,85.0,0.0,99.0,NaN
1,OEQjkMBoTqK7YmVc02NQDQ==,2018-10-25,17861-6,NaN,9.6,8.6,10.4,NaN
2,OEQjkMBoTqK7YmVc02NQDQ==,2018-10-25,33914-3,NaN,91.0,60.0,0.0,NaN
3,OEQjkMBoTqK7YmVc02NQDQ==,2018-10-25,4544-3,NaN,42.8,35.0,45.0,NaN
4,OEQjkMBoTqK7YmVc02NQDQ==,2018-10-25,713-8,NaN,6.8,0.0,0.0,NaN


In [55]:
list_values=df3['abnormal_code'].unique()
lst=[]
for i in list_values:
    lst.append((i, len(str(i))))
lst 

[(nan, 3),
 ('H', 1),
 ('L', 1),
 ('     ', 5),
 ('H    ', 5),
 ('L    ', 5),
 ('A', 1),
 ('A    ', 5),
 ('<', 1),
 ('-', 1),
 ('HH', 2),
 ('>', 1),
 ('LL', 2),
 ('<    ', 5),
 ('Y', 1),
 ('>    ', 5),
 ('+', 1),
 ('N    ', 5),
 ('Y    ', 5),
 ('NORMA', 5),
 ('HIGH ', 5),
 ('BELOW', 5),
 ('ABOVE', 5),
 ('N', 1),
 ('+    ', 5),
 ('*    ', 5),
 ('-    ', 5)]

In [56]:
df3['abnormal_code'].replace({'H    ': 'H', 'ABOVE': 'H', 'HIGH ':'H', '>    ': 'H', 'HH': 'H', '>': 'H', '+': 'H', '+    ': 'H'}, inplace=True)

df3['abnormal_code'].replace({'L    ': 'L', 'BELOW': 'L', '<    ': 'L', 'LL': 'L', '<': 'L', '-': 'L', '-    ': 'L'}, inplace=True)

df3['abnormal_code'].replace({'A    ': 'ABNORM', 'A': 'ABNORM', 'Y    ': 'ABNORM', 'Y': 'ABNORM', '*    ': 'ABNORM'}, inplace=True)

df3['abnormal_code'].replace({'NORMA': 'NORM', 'N': 'NORM', 'N    ': 'NORM'}, inplace=True)

df3['abnormal_code'].unique()


array([nan, 'H', 'L', '     ', 'ABNORM', 'NORM'], dtype=object)

We observed that some of the records where `abnormal_code` entered as a string with 5 white space are in fact results that must be entered as normal, and some others can't be determined since either lower bound or the upper bound is null. We will not update any of these values to keep the column with the original values. However, for the sake of readability we will change this string to 'DNE'  We will add another column called `abnormal_code_updated` where we update values accordingly.  

In [57]:
df3[df3['abnormal_code']=='     '].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 239541 entries, 55 to 5917885
Data columns (total 8 columns):
 #   Column                     Non-Null Count   Dtype         
---  ------                     --------------   -----         
 0   member_id                  239541 non-null  object        
 1   date_of_service            239541 non-null  datetime64[ns]
 2   loinc_code                 239541 non-null  object        
 3   abnormal_code              239541 non-null  object        
 4   result_value               193370 non-null  float64       
 5   normal_low_value_numeric   108243 non-null  float64       
 6   normal_high_value_numeric  108199 non-null  float64       
 7   abnormal_code_updated      239541 non-null  object        
dtypes: datetime64[ns](1), float64(3), object(4)
memory usage: 16.4+ MB


In [58]:
a=df3[df3['abnormal_code']=='     ']
a[(a['normal_low_value_numeric'].isna()) & (a['normal_high_value_numeric'].isna())].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 131294 entries, 55 to 5917883
Data columns (total 8 columns):
 #   Column                     Non-Null Count   Dtype         
---  ------                     --------------   -----         
 0   member_id                  131294 non-null  object        
 1   date_of_service            131294 non-null  datetime64[ns]
 2   loinc_code                 131294 non-null  object        
 3   abnormal_code              131294 non-null  object        
 4   result_value               85407 non-null   float64       
 5   normal_low_value_numeric   0 non-null       float64       
 6   normal_high_value_numeric  0 non-null       float64       
 7   abnormal_code_updated      131294 non-null  object        
dtypes: datetime64[ns](1), float64(3), object(4)
memory usage: 9.0+ MB


In [59]:
a=df3[df3['abnormal_code_updated']=='     ']
a[(a['normal_low_value_numeric'].notna()) & (a['normal_high_value_numeric'].notna())].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 108195 entries, 110 to 5917885
Data columns (total 8 columns):
 #   Column                     Non-Null Count   Dtype         
---  ------                     --------------   -----         
 0   member_id                  108195 non-null  object        
 1   date_of_service            108195 non-null  datetime64[ns]
 2   loinc_code                 108195 non-null  object        
 3   abnormal_code              108195 non-null  object        
 4   result_value               107920 non-null  float64       
 5   normal_low_value_numeric   108195 non-null  float64       
 6   normal_high_value_numeric  108195 non-null  float64       
 7   abnormal_code_updated      108195 non-null  object        
dtypes: datetime64[ns](1), float64(3), object(4)
memory usage: 7.4+ MB


In [60]:
a=df3[df3['abnormal_code_updated']=='     ']
a[(a['normal_low_value_numeric'].notna()) & (a['normal_high_value_numeric'].notna())].head()

,member_id,date_of_service,loinc_code,abnormal_code,result_value,normal_low_value_numeric,normal_high_value_numeric,abnormal_code_updated
110,OEQjkMBoTqK7YmVc02NQDQ==,2019-04-11,17861-6,,9.70,8.6,10.40,
111,OEQjkMBoTqK7YmVc02NQDQ==,2019-04-11,777-3,,228.00,140.0,400.00,
116,OEQjkMBoTqK7YmVc02NQDQ==,2019-04-11,1751-7,,4.20,3.6,5.10,
117,OEQjkMBoTqK7YmVc02NQDQ==,2019-04-11,3016-3,,4.11,0.4,4.50,
118,OEQjkMBoTqK7YmVc02NQDQ==,2019-04-11,2160-0,,0.68,0.5,0.99,


In [61]:
df3.loc[(df3['abnormal_code']=='     '),'abnormal_code']='DNE'
df3['abnormal_code'].unique() # check the update

array([nan, 'H', 'L', 'DNE', 'ABNORM', 'NORM'], dtype=object)

In [62]:
df3['abnormal_code'].value_counts(dropna=False)

NaN       4925154
H          443730
L          249975
DNE        239541
ABNORM      59967
NORM          245
Name: abnormal_code, dtype: int64

In [63]:
df3['abnormal_code_updated']=df3['abnormal_code']

If a result value lies in the reference interval we will label it as 'NORM' in the `abnormal_code_updated`. Similarly, a result value will be labelled as 'L' ('H') if it is strictly smaller (bigger) than the lower bound (upper bound).

In [64]:
df3.loc[(df3['normal_low_value_numeric']<= df3['result_value']) & (df3['result_value']<=df3['normal_high_value_numeric']),'abnormal_code_updated']='NORM'

df3.loc[(df3['result_value']< df3['normal_low_value_numeric']), 'abnormal_code_updated']='L'

df3.loc[(df3['normal_high_value_numeric']!=0.0) & (df3['result_value']> df3['normal_high_value_numeric']), 'abnormal_code_updated']='H'

In [65]:
df3['abnormal_code_updated'].value_counts(dropna=False)

NORM      3378798
NaN       1653917
H          443858
L          250496
DNE        131583
ABNORM      59960
Name: abnormal_code_updated, dtype: int64

Now notice that we have have no way of knowing the actual abnormal code for the remaining records entered as DNE, therefore we will convert them into null. 

In [66]:
df3.loc[df3['abnormal_code_updated']=='DNE', 'abnormal_code_updated']=np.nan

In [67]:
df3['abnormal_code_updated'].value_counts(dropna=False)

NORM      3378798
NaN       1785500
H          443858
L          250496
ABNORM      59960
Name: abnormal_code_updated, dtype: int64

In [68]:
df3.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5918612 entries, 0 to 5918611
Data columns (total 8 columns):
 #   Column                     Non-Null Count    Dtype         
---  ------                     --------------    -----         
 0   member_id                  5918612 non-null  object        
 1   date_of_service            5918612 non-null  datetime64[ns]
 2   loinc_code                 5918612 non-null  object        
 3   abnormal_code              993458 non-null   object        
 4   result_value               4977832 non-null  float64       
 5   normal_low_value_numeric   5124027 non-null  float64       
 6   normal_high_value_numeric  5257167 non-null  float64       
 7   abnormal_code_updated      4133112 non-null  object        
dtypes: datetime64[ns](1), float64(3), object(4)
memory usage: 361.2+ MB


Next, we will copy `abnormal_code_updated`  and update the values abnormal vs normal. If a value is 'H', or 'L' it will be updated to 'ABNORM'.

In [69]:
df3['abnormal_vs_normal']=df3['abnormal_code_updated']
df3.head() #check the update

,member_id,date_of_service,loinc_code,abnormal_code,result_value,normal_low_value_numeric,normal_high_value_numeric,abnormal_code_updated,abnormal_vs_normal
0,OEQjkMBoTqK7YmVc02NQDQ==,2018-10-25,13457-7,NaN,85.0,0.0,99.0,NORM,NORM
1,OEQjkMBoTqK7YmVc02NQDQ==,2018-10-25,17861-6,NaN,9.6,8.6,10.4,NORM,NORM
2,OEQjkMBoTqK7YmVc02NQDQ==,2018-10-25,33914-3,NaN,91.0,60.0,0.0,NaN,NaN
3,OEQjkMBoTqK7YmVc02NQDQ==,2018-10-25,4544-3,NaN,42.8,35.0,45.0,NORM,NORM
4,OEQjkMBoTqK7YmVc02NQDQ==,2018-10-25,713-8,NaN,6.8,0.0,0.0,NaN,NaN


In [70]:
df3['abnormal_vs_normal'].replace({'H': 'ABNORM', 'L': 'ABNORM'}, inplace=True)
df3['abnormal_vs_normal'].value_counts(dropna=False)

NORM      3378798
NaN       1785500
ABNORM     754314
Name: abnormal_vs_normal, dtype: int64

In [71]:
df3.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5918612 entries, 0 to 5918611
Data columns (total 9 columns):
 #   Column                     Non-Null Count    Dtype         
---  ------                     --------------    -----         
 0   member_id                  5918612 non-null  object        
 1   date_of_service            5918612 non-null  datetime64[ns]
 2   loinc_code                 5918612 non-null  object        
 3   abnormal_code              993458 non-null   object        
 4   result_value               4977832 non-null  float64       
 5   normal_low_value_numeric   5124027 non-null  float64       
 6   normal_high_value_numeric  5257167 non-null  float64       
 7   abnormal_code_updated      4133112 non-null  object        
 8   abnormal_vs_normal         4133112 non-null  object        
dtypes: datetime64[ns](1), float64(3), object(5)
memory usage: 406.4+ MB


In [72]:
df3.drop(['date_of_service'], axis=1, inplace=True)
df3.head(3)# check the update

,member_id,loinc_code,abnormal_code,result_value,normal_low_value_numeric,normal_high_value_numeric,abnormal_code_updated,abnormal_vs_normal
0,OEQjkMBoTqK7YmVc02NQDQ==,13457-7,NaN,85.0,0.0,99.0,NORM,NORM
1,OEQjkMBoTqK7YmVc02NQDQ==,17861-6,NaN,9.6,8.6,10.4,NORM,NORM
2,OEQjkMBoTqK7YmVc02NQDQ==,33914-3,NaN,91.0,60.0,0.0,NaN,NaN


We will drop `abnormal_code`,`result_value`,`normal_low_value_numeric`, `normal_high_value_numeric`,`abnormal_vs_normal` since `abnormal_code_updated` will capture the information we need.

In [73]:
df3.drop(['abnormal_code', 'result_value','normal_low_value_numeric', 'normal_high_value_numeric', 'abnormal_vs_normal'], axis=1, inplace=True)
df3.head() # check the update

,member_id,loinc_code,abnormal_code_updated
0,OEQjkMBoTqK7YmVc02NQDQ==,13457-7,NORM
1,OEQjkMBoTqK7YmVc02NQDQ==,17861-6,NORM
2,OEQjkMBoTqK7YmVc02NQDQ==,33914-3,NaN
3,OEQjkMBoTqK7YmVc02NQDQ==,4544-3,NORM
4,OEQjkMBoTqK7YmVc02NQDQ==,713-8,NaN


In [74]:
# save the data to a new csv file
datapath = '../Outputs'
save_file(df3, 'df3_final.csv', datapath)

A file already exists with this name.

Do you want to overwrite? (Y/N)Y
Writing file.  "../Outputs/df3_final.csv"


### 2.7.4 Patients Data<a id='2.7.4_Patients_Data'></a>

In [75]:
df4.head(5)

,member_id,patient_age,patient_gender,dual_status,insurance_company,insurance_type,pbp_type,county,city
0,AAHLhKW7RYOEQIemWrh/1w==,72,M,DYoOR0bMMk3HRSo8sSPjUw==,GUD2qR6A4Wd2L73mE5cXSw==,0EM/AFr1Il2Z8J30Yadekw==,0EM/AFr1Il2Z8J30Yadekw==,UejZIiyN0zMFVlUbJl9WMw==,JcyeVCRLiLD21PlyRx0v3w==
1,AAKW/PKhRva+ZS2B4s0AmA==,72,M,tMEZlLbu7jT3s3qXZC+gWQ==,rOCadimmYUC//TJHZzkvRg==,0EM/AFr1Il2Z8J30Yadekw==,a85jcyRlCtomarSMXXjFbg==,1S4k4hzgPOiWaL9v6ChejA==,btsrhf68Jq+yI/2V8QTMMw==
2,AAT/v3ExQAyfas8Ov5JevA==,75,F,tMEZlLbu7jT3s3qXZC+gWQ==,rOCadimmYUC//TJHZzkvRg==,0EM/AFr1Il2Z8J30Yadekw==,9Ro3ya9jPvS3mg+7vLpsgg==,1S4k4hzgPOiWaL9v6ChejA==,btsrhf68Jq+yI/2V8QTMMw==
3,AAoq6W9QR0G/kOfNZnoCPg==,62,M,cxTcUuoRUVTg+SSt7OlnXg==,hdbB94EoBNp2gZ5k2t6eLw==,0EM/AFr1Il2Z8J30Yadekw==,9Ro3ya9jPvS3mg+7vLpsgg==,EvFxDbLnPKOa13Yw9vDU4A==,xg5tPYYDxhRt10rlHSQG3A==
4,AAu2+UMHSpK6gVYffGi4pw==,54,F,tMEZlLbu7jT3s3qXZC+gWQ==,rOCadimmYUC//TJHZzkvRg==,0EM/AFr1Il2Z8J30Yadekw==,0EM/AFr1Il2Z8J30Yadekw==,EvFxDbLnPKOa13Yw9vDU4A==,yjYhcMttIqpWhMY2PuP8Sg==


In [76]:
df4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26559 entries, 0 to 26558
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   member_id          26559 non-null  object
 1   patient_age        26559 non-null  int64 
 2   patient_gender     26559 non-null  object
 3   dual_status        26559 non-null  object
 4   insurance_company  26559 non-null  object
 5   insurance_type     26559 non-null  object
 6   pbp_type           26558 non-null  object
 7   county             26547 non-null  object
 8   city               26552 non-null  object
dtypes: int64(1), object(8)
memory usage: 1.8+ MB


In [77]:
df4.nunique()

member_id            26559
patient_age             83
patient_gender           2
dual_status              9
insurance_company       12
insurance_type           2
pbp_type                10
county                  48
city                   499
dtype: int64

We need to change most values in 'df4' for readability purposes, as seen below.

In [78]:
df4['dual_status'].unique()

array(['DYoOR0bMMk3HRSo8sSPjUw==', 'tMEZlLbu7jT3s3qXZC+gWQ==',
       'cxTcUuoRUVTg+SSt7OlnXg==', 'D7HF1L7pKm0+yiZHnui7hQ==',
       'K3iVxpF0tRf+h5l85tlfpA==', 'ssNL6tTkjkgJfijNgtfX1Q==',
       'GwdXg8Z44j4sOkS6HTTM3A==', 'MKqEIzoTiCq5E7Rp2PAR0w==',
       'xTNihcc8vr7yvMoWJBWwvA=='], dtype=object)

We will next create some dictionaries for the new values of 'df4'.

In [79]:
#-------------------------------------------------------------------------

list_new_values_df4=[]

#-------------------------------------------------------------------------

dict_new_values_dual_status={} 
a=0
for item in df4['dual_status'].unique():
    a=a+1
    dict_new_values_dual_status[item]=str(a)
list_new_values_df4.append(dict_new_values_dual_status)

#-------------------------------------------------------------------------

dict_new_values_insurance_company={}
a=0
for item in df4['insurance_company'].unique():
    a=a+1
    dict_new_values_insurance_company[item]=str(a)
list_new_values_df4.append(dict_new_values_insurance_company)

#-------------------------------------------------------------------------

dict_new_values_insurance_type={}
a=0
for item in df4['insurance_type'].unique():
    a=a+1
    dict_new_values_insurance_type[item]=str(a)
list_new_values_df4.append(dict_new_values_insurance_type)

#-------------------------------------------------------------------------

dict_new_values_pbp_type={}
a=0
for item in df4['pbp_type'].unique():
    a=a+1
    dict_new_values_pbp_type[item]=str(a)
list_new_values_df4.append(dict_new_values_pbp_type)    

#-------------------------------------------------------------------------

dict_new_values_county={}
a=0
for item in df4['county'].unique():
    a=a+1
    dict_new_values_county[item]=str(a)
list_new_values_df4.append(dict_new_values_county)    


#-------------------------------------------------------------------------

dict_new_values_city={}
a=0
for item in df4['city'].unique():
    a=a+1
    dict_new_values_city[item]=str(a)
list_new_values_df4.append(dict_new_values_city)    

#-------------------------------------------------------------------------

#list_new_values_df4

Values of all columns of 'df4' will be updated except `patient_age` and `patient_gender`.

In [80]:
df4['dual_status'].replace(dict_new_values_dual_status, inplace=True)

df4['insurance_company'].replace(dict_new_values_insurance_company, inplace=True)

df4['insurance_type'].replace(dict_new_values_insurance_type, inplace=True)

df4['pbp_type'].replace(dict_new_values_pbp_type, inplace=True)

df4['county'].replace(dict_new_values_county, inplace=True)

df4['city'].replace(dict_new_values_city, inplace=True)

df4.head() # check the update

,member_id,patient_age,patient_gender,dual_status,insurance_company,insurance_type,pbp_type,county,city
0,AAHLhKW7RYOEQIemWrh/1w==,72,M,1,1,1,1,1,1
1,AAKW/PKhRva+ZS2B4s0AmA==,72,M,2,2,1,2,2,2
2,AAT/v3ExQAyfas8Ov5JevA==,75,F,2,2,1,3,2,2
3,AAoq6W9QR0G/kOfNZnoCPg==,62,M,3,3,1,3,3,3
4,AAu2+UMHSpK6gVYffGi4pw==,54,F,2,2,1,1,3,4


Last we will change the name of `patient_age` and `patient_gender`. 

In [81]:
df4.rename(columns={'patient_age': 'age', 'patient_gender': 'sex'}, inplace=True)
df4.head(3) # check the update

,member_id,age,sex,dual_status,insurance_company,insurance_type,pbp_type,county,city
0,AAHLhKW7RYOEQIemWrh/1w==,72,M,1,1,1,1,1,1
1,AAKW/PKhRva+ZS2B4s0AmA==,72,M,2,2,1,2,2,2
2,AAT/v3ExQAyfas8Ov5JevA==,75,F,2,2,1,3,2,2


In [82]:
# save the data to a new csv file
datapath = '../Outputs'
save_file(df4, 'df4_final.csv', datapath)

A file already exists with this name.

Do you want to overwrite? (Y/N)Y
Writing file.  "../Outputs/df4_final.csv"


### 2.7.5 Prescriptions Data<a id='2.7.5_Prescriptions_Data'></a>

In [83]:
df5.head(3)

,member_id,date_filled,ndc_number,days_supply,drug_name,metric_quantity,new_or_refill
0,AAHLhKW7RYOEQIemWrh/1w==,2018-01-13,60505013400,60.0,CYCLOSPORINE 100 MG CAPS@100 M,60,N
1,AAHLhKW7RYOEQIemWrh/1w==,2018-01-16,62175013643,90.0,OMEPRAZOLE DR 40 MG CAPS@40 MG,90,N
2,AAHLhKW7RYOEQIemWrh/1w==,2018-02-06,50742013510,90.0,ALLOPURINOL 100 MG TABLE@100 M,90,N


In [84]:
df5.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1740855 entries, 0 to 1740854
Data columns (total 7 columns):
 #   Column           Non-Null Count    Dtype  
---  ------           --------------    -----  
 0   member_id        1740855 non-null  object 
 1   date_filled      1740855 non-null  object 
 2   ndc_number       1732568 non-null  object 
 3   days_supply      1720363 non-null  float64
 4   drug_name        1738236 non-null  object 
 5   metric_quantity  1740855 non-null  int64  
 6   new_or_refill    1290968 non-null  object 
dtypes: float64(1), int64(1), object(5)
memory usage: 93.0+ MB


In [85]:
df5.nunique()

member_id          26090
date_filled          730
ndc_number         15915
days_supply          360
drug_name          10297
metric_quantity      648
new_or_refill          2
dtype: int64

We convert `date_filled` into datetime dtype. We will also check years of the services to eliminate any year other than 2018 and 2019.

In [86]:
df5['date_filled']=pd.to_datetime(df5['date_filled'])
df5.info() # check the update

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1740855 entries, 0 to 1740854
Data columns (total 7 columns):
 #   Column           Dtype         
---  ------           -----         
 0   member_id        object        
 1   date_filled      datetime64[ns]
 2   ndc_number       object        
 3   days_supply      float64       
 4   drug_name        object        
 5   metric_quantity  int64         
 6   new_or_refill    object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(4)
memory usage: 93.0+ MB


In [87]:
pd.DatetimeIndex(df5['date_filled']).year.unique()

Int64Index([2018, 2019], dtype='int64', name='date_filled')

In [88]:
df5.describe()

,days_supply,metric_quantity
count,1.720363e+06,1.740855e+06
mean,5.301320e+01,7.891937e+01
std,4.598313e+01,1.815798e+02
min,-5.450000e+02,-7.500000e+03
25%,2.800000e+01,2.800000e+01
50%,4.500000e+01,9.000000e+01
75%,9.000000e+01,9.000000e+01
max,9.990000e+02,2.430000e+04


In [89]:
df5['member_id'].value_counts()

gkCu8pm2RKalgzZod7DsMw==    820
bLFxKbJ4TriTMvxbMK1wLQ==    657
UzDCsdfZRNG3HcQ3H1oJmA==    601
3rtr5jzqTHKCkD+Ibb3MTQ==    585
tqv0K291SoidlMklCGDYGA==    580
                           ... 
3FxbW6wqRoq1c//b5sqJNQ==      1
3LAM+HWMR5qyP2dfFslF/w==      1
p7E5nBOtR5iwpmlQ8btfNw==      1
xH4cw1GaTP+nRhem4zZ83g==      1
ZwI456tWT12l0YRBC1QHZA==      1
Name: member_id, Length: 26090, dtype: int64

We will create a column called `drug_quantity_rate` to record `days_supply` divided by `metric_quantity`. This will indicate how much medicine was given to the patient for each medicine prescribed. 

In [90]:
df5['drug_quantity_rate']=df5['days_supply']/df5['metric_quantity']

The column `ndc_number`contains the unique drug id which is sufficient to identify prescribed medicine for each patient. We will drop `drug_name` since it is no longer needed. We will also drop `date_filled` column since we will not consider the dates in our model.

In [91]:
df5.drop(['drug_name', 'date_filled'], axis=1, inplace=True)
df5.head(3) # check the update

,member_id,ndc_number,days_supply,metric_quantity,new_or_refill,drug_quantity_rate
0,AAHLhKW7RYOEQIemWrh/1w==,60505013400,60.0,60,N,1.0
1,AAHLhKW7RYOEQIemWrh/1w==,62175013643,90.0,90,N,1.0
2,AAHLhKW7RYOEQIemWrh/1w==,50742013510,90.0,90,N,1.0


Last we will drop `days_supply` and `metric_quantity` since we will record the quantity using the column `drug_quantity_rate`.

In [92]:
df5.drop(['days_supply', 'metric_quantity'], axis=1, inplace=True)
df5.head(3) # check the update

,member_id,ndc_number,new_or_refill,drug_quantity_rate
0,AAHLhKW7RYOEQIemWrh/1w==,60505013400,N,1.0
1,AAHLhKW7RYOEQIemWrh/1w==,62175013643,N,1.0
2,AAHLhKW7RYOEQIemWrh/1w==,50742013510,N,1.0


In [93]:
# save the data to a new csv file
datapath = '../Outputs'
save_file(df5, 'df5_final.csv', datapath)

A file already exists with this name.

Do you want to overwrite? (Y/N)Y
Writing file.  "../Outputs/df5_final.csv"


### 2.7.6 Train Data<a id='2.7.6_Train_Data'></a>

In [94]:
df6.head(3)

,member_id,hcc_9,hcc_10,hcc_11,hcc_12,hcc_18,hcc_19,hcc_21,hcc_22,hcc_23,...,hcc_88,hcc_96,hcc_103,hcc_108,hcc_111,hcc_112,hcc_124,hcc_137,hcc_138,hcc_161
0,c5xNoBb+QoCII8LRiNnScg==,0,0,0,0,0,0,0,1,0,...,0,1,0,1,1,0,0,0,1,0
1,rofIXF8fT9CAAtouT8yPSQ==,0,0,0,0,1,0,0,0,0,...,1,1,0,0,0,0,0,1,1,0
2,7Rs4fvk6TJG2rYLiXGLFnQ==,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [95]:
df6.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26559 entries, 0 to 26558
Data columns (total 31 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   member_id  26559 non-null  object
 1   hcc_9      26559 non-null  int64 
 2   hcc_10     26559 non-null  int64 
 3   hcc_11     26559 non-null  int64 
 4   hcc_12     26559 non-null  int64 
 5   hcc_18     26559 non-null  int64 
 6   hcc_19     26559 non-null  int64 
 7   hcc_21     26559 non-null  int64 
 8   hcc_22     26559 non-null  int64 
 9   hcc_23     26559 non-null  int64 
 10  hcc_40     26559 non-null  int64 
 11  hcc_47     26559 non-null  int64 
 12  hcc_48     26559 non-null  int64 
 13  hcc_51     26559 non-null  int64 
 14  hcc_52     26559 non-null  int64 
 15  hcc_55     26559 non-null  int64 
 16  hcc_59     26559 non-null  int64 
 17  hcc_75     26559 non-null  int64 
 18  hcc_79     26559 non-null  int64 
 19  hcc_84     26559 non-null  int64 
 20  hcc_85     26559 non-null  i

In [96]:
df6.nunique()

member_id    26559
hcc_9            2
hcc_10           2
hcc_11           2
hcc_12           2
hcc_18           2
hcc_19           2
hcc_21           2
hcc_22           2
hcc_23           2
hcc_40           2
hcc_47           2
hcc_48           2
hcc_51           2
hcc_52           2
hcc_55           2
hcc_59           2
hcc_75           2
hcc_79           2
hcc_84           2
hcc_85           2
hcc_88           2
hcc_96           2
hcc_103          2
hcc_108          2
hcc_111          2
hcc_112          2
hcc_124          2
hcc_137          2
hcc_138          2
hcc_161          2
dtype: int64

### Part B <a id='Part_B'></a> 

### 2.8 Subset the Data<a id='2.8_Subset_the_Data'></a>

* The size of our data is very large. This can cause some issues with the implementations of our algorithms. In this part, we will subset all the dataframes to make sure that we will not run into an issue.

* We will filter the data with respect to the patients who have illnesses 'I10' and/or 'I739' or neither. Therefore our goal is to classify patients based on whether they have one, both, or neither of these illnesses. 

### 2.8.1 Subset Disease Data<a id='2.8.1_Subset_Disease_Data'></a>

In [97]:
df2['icd_code'].value_counts()

I10        29172
I739       18791
E1151      18603
J449       16953
E785       14982
           ...  
Z965           1
E0969          1
M84433A        1
S52691A        1
M80051S        1
Name: icd_code, Length: 12951, dtype: int64

In [98]:
df2[df2['icd_code']=='I10'].nunique()

member_id               19940
icd_code                    1
icd_chronic_or_acute        1
dtype: int64

In [99]:
df2[df2['icd_code']=='I739'].nunique()

member_id               13404
icd_code                    1
icd_chronic_or_acute        1
dtype: int64

In [100]:
df2_I10_I739=df2[(df2['icd_code']=='I10') | (df2['icd_code']=='I739')]
df2_I10_I739.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 47963 entries, 0 to 1022148
Data columns (total 3 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   member_id             47963 non-null  object
 1   icd_code              47963 non-null  object
 2   icd_chronic_or_acute  47963 non-null  object
dtypes: object(3)
memory usage: 1.5+ MB


In [101]:
df2_I10_I739.head()

,member_id,icd_code,icd_chronic_or_acute
0,AAHLhKW7RYOEQIemWrh/1w==,I739,Chronic
5,AAHLhKW7RYOEQIemWrh/1w==,I739,Chronic
36,AAKW/PKhRva+ZS2B4s0AmA==,I10,Unknown
45,AAKW/PKhRva+ZS2B4s0AmA==,I739,Chronic
106,AAlNgHwPRw2nNdCnsEKnyA==,I10,Unknown


In [102]:
df2_neither_I10_nor_I739=df2[(df2['icd_code']!='I10') & (df2['icd_code']!='I739')]
df2_neither_I10_nor_I739.head()

,member_id,icd_code,icd_chronic_or_acute
1,AAHLhKW7RYOEQIemWrh/1w==,M545,Unknown
2,AAHLhKW7RYOEQIemWrh/1w==,N183,Chronic
3,AAHLhKW7RYOEQIemWrh/1w==,E6601,Chronic
4,AAHLhKW7RYOEQIemWrh/1w==,J8410,Chronic
6,AAHLhKW7RYOEQIemWrh/1w==,L4050,Chronic


In [103]:
df2_I10_I739_subset=df2_I10_I739.sample(n=2000, random_state=12)


In [104]:
df2_neither_I10_nor_I739_subset=df2_neither_I10_nor_I739.sample(n=1000, random_state=24)
df2_neither_I10_nor_I739_subset.nunique()

member_id               973
icd_code                505
icd_chronic_or_acute      2
dtype: int64

In [105]:
df2_neither_I10_nor_I739_subset.head()

,member_id,icd_code,icd_chronic_or_acute
138401,Ijv6AOBzRkOayI4etpUHNg==,R001,Unknown
595281,lElruXemQ+GpMN8bO5JFmQ==,I5022,Chronic
438652,bSd+vF+sQiasYTeTPei1WQ==,E041,Unknown
162828,KHtFM1ZURCitgf1nBedVQw==,E1151,Chronic
684767,qn1yWe2iTr2hj2W91DwxLg==,R739,Unknown


In [106]:
df2_neither_I10_nor_I739_subset['icd_code']='neither'
df2_neither_I10_nor_I739_subset.head()

,member_id,icd_code,icd_chronic_or_acute
138401,Ijv6AOBzRkOayI4etpUHNg==,neither,Unknown
595281,lElruXemQ+GpMN8bO5JFmQ==,neither,Chronic
438652,bSd+vF+sQiasYTeTPei1WQ==,neither,Unknown
162828,KHtFM1ZURCitgf1nBedVQw==,neither,Chronic
684767,qn1yWe2iTr2hj2W91DwxLg==,neither,Unknown


In [107]:
df2_subset=pd.concat([df2_I10_I739_subset,df2_neither_I10_nor_I739_subset ])
df2_subset.nunique()

member_id               2842
icd_code                   3
icd_chronic_or_acute       2
dtype: int64

In [108]:
a=df2_subset[df2_subset['icd_code']=='I10']['member_id']
b=df2_subset[df2_subset['icd_code']=='I739']['member_id']
len(set(a).intersection(set(b)))

29

In [109]:
df2_subset['icd_code'].value_counts()

I10        1233
neither    1000
I739        767
Name: icd_code, dtype: int64

### 2.8.2 Subset Admissions Data<a id='2.8.2_Subset_Admissions_Data'></a>

In [110]:
df1_subset=df1[df1['member_id'].isin(df2_subset['member_id'])]

In [111]:
df1_subset['member_id'].value_counts()

3LEtcGKvR964GbMQrmOtAQ==    41
2nPEQV1GRB+Qa0RVvzmvgA==    24
Bz9tfw7BRxKLOSNubZ6LZA==    24
LyMK/mSlTXS3C1kUqyA0ng==    24
ieyPtRprQxu3bqDUTn8GPg==    20
                            ..
hFuUp9gWSdap66KZDJkw7A==     1
WASM0ASgQYOH+DvqFvE5wg==     1
TUCuqazwS/6noTFSyXgPIg==     1
1wwAFJXHSaae7wOvqU6RuQ==     1
dCNookl8R6mDKIogYpAEnw==     1
Name: member_id, Length: 1137, dtype: int64

We will remove the member id that shows up 41 times in all dataframes since this is an outlier (see above cell).

In [112]:
df1_subset=df1_subset[~(df1_subset['member_id']=='3LEtcGKvR964GbMQrmOtAQ==')]
df1_subset['member_id'].value_counts() # check the update

LyMK/mSlTXS3C1kUqyA0ng==    24
2nPEQV1GRB+Qa0RVvzmvgA==    24
Bz9tfw7BRxKLOSNubZ6LZA==    24
ieyPtRprQxu3bqDUTn8GPg==    20
E3UDCgfGTzO/1+vKcb12tQ==    18
                            ..
Ayok9jOETJq8z1VzpBuY7A==     1
EmNCXBc3Q8+nQy5KetkMMg==     1
EN7I7oktTRWJnOPaFO6xkg==     1
9xopV+I8Qci7ZiCz/jGhAw==     1
Qdy+MT5nQwitlQK4m+E9Cg==     1
Name: member_id, Length: 1136, dtype: int64

In [113]:
df2_subset=df2_subset[~(df2_subset['member_id']=='3LEtcGKvR964GbMQrmOtAQ==')]
df2_subset[df2_subset['member_id']=='3LEtcGKvR964GbMQrmOtAQ=='] # check the update

,member_id,icd_code,icd_chronic_or_acute


In [114]:
# save the data to a new csv file
datapath = '../Outputs'
save_file(df1_subset, 'df1_subset.csv', datapath)

A file already exists with this name.

Do you want to overwrite? (Y/N)Y
Writing file.  "../Outputs/df1_subset.csv"


### 2.8.3 Subset Labs Data<a id='2.8.3_Subset_Labs_Data'></a>

In [115]:
df3_subset=df3[df3['member_id'].isin(df2_subset['member_id'])]
df3_subset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 620077 entries, 531 to 5918611
Data columns (total 3 columns):
 #   Column                 Non-Null Count   Dtype 
---  ------                 --------------   ----- 
 0   member_id              620077 non-null  object
 1   loinc_code             620077 non-null  object
 2   abnormal_code_updated  432566 non-null  object
dtypes: object(3)
memory usage: 18.9+ MB


In [116]:
df3_subset.nunique()

member_id                2290
loinc_code               1659
abnormal_code_updated       4
dtype: int64

In [117]:
df3_subset=df3_subset[~(df3_subset['member_id']=='3LEtcGKvR964GbMQrmOtAQ==')]
df3_subset[df3_subset['member_id']=='3LEtcGKvR964GbMQrmOtAQ=='] # check the update

,member_id,loinc_code,abnormal_code_updated


### 2.8.4 Subset Patients Data<a id='2.8.4_Subset_Patients_Data'></a>

In [118]:
df4_subset=df4[df4['member_id'].isin(df2_subset['member_id'])]
df4_subset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2444 entries, 0 to 26558
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   member_id          2444 non-null   object
 1   age                2444 non-null   int64 
 2   sex                2444 non-null   object
 3   dual_status        2444 non-null   object
 4   insurance_company  2444 non-null   object
 5   insurance_type     2444 non-null   object
 6   pbp_type           2444 non-null   object
 7   county             2444 non-null   object
 8   city               2444 non-null   object
dtypes: int64(1), object(8)
memory usage: 190.9+ KB


In [119]:
df4_subset=df4_subset[~(df4_subset['member_id']=='3LEtcGKvR964GbMQrmOtAQ==')]
df4_subset[df4_subset['member_id']=='3LEtcGKvR964GbMQrmOtAQ=='] # check the update

,member_id,age,sex,dual_status,insurance_company,insurance_type,pbp_type,county,city


### 2.8.5 Subset Prescriptions Data<a id='2.8.5_Subset_Prescriptions_Data'></a>

In [120]:
df5_subset=df5[df5['member_id'].isin(df2_subset['member_id'])]
df5_subset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 179827 entries, 0 to 1740854
Data columns (total 4 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   member_id           179827 non-null  object 
 1   ndc_number          179423 non-null  object 
 2   new_or_refill       139794 non-null  object 
 3   drug_quantity_rate  168040 non-null  float64
dtypes: float64(1), object(3)
memory usage: 6.9+ MB


In [121]:
df5_subset.nunique()

member_id             2410
ndc_number            7483
new_or_refill            2
drug_quantity_rate    1074
dtype: int64

In [122]:
df5_subset=df5_subset[~(df5_subset['member_id']=='3LEtcGKvR964GbMQrmOtAQ==')]
df5_subset[df5_subset['member_id']=='3LEtcGKvR964GbMQrmOtAQ=='] # check the update

,member_id,ndc_number,new_or_refill,drug_quantity_rate


### 2.9 One Record per Member Id<a id='2.9_One_Record_per_Member_Id'></a>

In this section we will convert each dataframe into a new dataframe where we have exactly one record per patient/member id. 

### 2.9.1 Admissions Data Converted<a id='2.9.1_Admissions_Data'></a>

In [123]:
df1_subset.head(10)

,member_id,admission_type,is_readmission,er_to_inp_admission_transfer,days_to_prev_admission,visit_duration,visit_duration_updated,nth_visit
17,ADD7uXD9TBOUjJD43H3tNw==,ER,No,N,NaN,0,1,1
18,ADD7uXD9TBOUjJD43H3tNw==,ER,No,N,45.0,0,1,2
19,ADD7uXD9TBOUjJD43H3tNw==,ER,No,N,47.0,0,1,3
53,AH8v69t8QBW1u2Iuh/5qlQ==,ER,No,Y,NaN,0,1,1
54,AH8v69t8QBW1u2Iuh/5qlQ==,INP,Yes,Y,NaN,0,1,2
55,AIEq/y06TGOCCjwkDzmaeA==,INP,Yes,Y,NaN,2,3,1
56,AIEq/y06TGOCCjwkDzmaeA==,ER,Yes,Y,NaN,2,3,2
57,AIEq/y06TGOCCjwkDzmaeA==,INP,No,N,NaN,2,3,3
58,AInQlUfTRjSpklIOiuCifg==,ER,No,N,NaN,0,1,1
64,AK/yuahPR3Gx6dUrPtptzw==,OUT,No,N,NaN,0,1,1


To have one row per patient we will use two functions that we wrote; `df1_one_record_per_member_id()` and `fill_df1_res()`, see the auxiliary_functions.ipynb.  We consider each visit without any aggregation. The maximum number of visits among all patients is 24 (see below). 

 * We will create 24 columms called `v1`, `v2`, ..., `v_24` where we keep the records whether the patient had a visit or not. For instance, a patient with only 5 visits will have 1 for all columns `v1` through `v5` and nulls in all other columns `v6` through`v_24`. 

* We will also have the columns `vi_re_adm`, `vi_er_to_inp`, `vi_days_to_prev`, `vi_len`, `vi_len_pos`, `vi_ER`,..., `vi_INP REHAB` for each i=1,2,...,24 to include all the information for ith visit.  
 

In [124]:
df1_res=df1_one_record_per_member_id(df1_subset)

In [125]:
df1_res.head()

,member_id,num_of_visit,v1,v1_re_adm,v1_er_to_inp,v1_days_to_prev,v1_len,v1_len_pos,v1_ER,v1_INP,...,v24_len,v24_len_pos,v24_ER,v24_INP,v24_OUT,v24_OTH,v24_OBS,v24_SNF,v24_INP REHAB,v24_HOSPICE
0,ADD7uXD9TBOUjJD43H3tNw==,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,AH8v69t8QBW1u2Iuh/5qlQ==,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,AIEq/y06TGOCCjwkDzmaeA==,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,AInQlUfTRjSpklIOiuCifg==,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,AK/yuahPR3Gx6dUrPtptzw==,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Now we will fill 'df1_res' accordingly using the function 'fill_df1_res()'.

In [126]:
df1_res=fill_df1_res(df1_res, df1_subset)

In [127]:
df1_res.head()

,member_id,num_of_visit,v1,v1_re_adm,v1_er_to_inp,v1_days_to_prev,v1_len,v1_len_pos,v1_ER,v1_INP,...,v24_len,v24_len_pos,v24_ER,v24_INP,v24_OUT,v24_OTH,v24_OBS,v24_SNF,v24_INP REHAB,v24_HOSPICE
0,ADD7uXD9TBOUjJD43H3tNw==,3,1,0,0,NaN,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
1,AH8v69t8QBW1u2Iuh/5qlQ==,2,1,0,1,NaN,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
2,AIEq/y06TGOCCjwkDzmaeA==,3,1,1,1,NaN,2,3,0,1,...,0,0,0,0,0,0,0,0,0,0
3,AInQlUfTRjSpklIOiuCifg==,1,1,0,0,NaN,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
4,AK/yuahPR3Gx6dUrPtptzw==,8,1,0,0,NaN,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [128]:
df1_res.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1136 entries, 0 to 1135
Columns: 338 entries, member_id to v24_HOSPICE
dtypes: float64(14), int64(323), object(1)
memory usage: 2.9+ MB


In [129]:
# save the data to a new csv file
datapath = '../Outputs'
save_file(df1_res, 'df1_res.csv', datapath)

A file already exists with this name.

Do you want to overwrite? (Y/N)Y
Writing file.  "../Outputs/df1_res.csv"


In [130]:
df1_res.nunique() # check df1_res

member_id        1136
num_of_visit       19
v1                  1
v1_re_adm           2
v1_er_to_inp        2
                 ... 
v24_OTH             1
v24_OBS             1
v24_SNF             1
v24_INP REHAB       2
v24_HOSPICE         1
Length: 338, dtype: int64

In [131]:
df1_res[df1_res['member_id']=='AH8v69t8QBW1u2Iuh/5qlQ==']['v1_ER']
# check the df1_res

1    1
Name: v1_ER, dtype: int64

In [132]:
df1_res[df1_res['member_id']=='AIEq/y06TGOCCjwkDzmaeA==']['v3_ER']
# check df1_res

2    0
Name: v3_ER, dtype: int64

### 2.9.2 Disease Data Converted<a id='2.9.2_Disease_Data'></a>

In [133]:
df2_subset.head()

,member_id,icd_code,icd_chronic_or_acute
636201,nr+43zfHQPKSwx2IJuJI5Q==,I10,Unknown
351311,Vwf7mI0tTGC4amYJPD6uJg==,I739,Chronic
310891,TRudRczSQt6dwb6EeZ1RLA==,I739,Chronic
347932,VjG87+cBSB2B1+loMmoHCg==,I739,Chronic
943332,7CfP6Hq5Qy6J0rXIuTc7kw==,I10,Unknown


In [134]:
df2_subset['member_id'].unique()

array(['nr+43zfHQPKSwx2IJuJI5Q==', 'Vwf7mI0tTGC4amYJPD6uJg==',
       'TRudRczSQt6dwb6EeZ1RLA==', ..., '71rJt0vGQTaGp1yQ9jBB/g==',
       'lzOoZy24Qc2CxCC1i1wsEA==', '3uE+j85IT8yqQhaDQ2Pe7Q=='],
      dtype=object)

In [135]:
# save the data to a new csv file
datapath = '../Outputs'
save_file(df2_subset, 'df2_subset.csv', datapath)

A file already exists with this name.

Do you want to overwrite? (Y/N)Y
Writing file.  "../Outputs/df2_subset.csv"


In [136]:
df2_res=df2_one_record_per_member_id(df2_subset)

In [137]:
df2_res.head()

,member_id,icd_I10,icd_I10_chronic,icd_I739,icd_I739_chronic,icd_neither,icd_neither_chronic
0,nr+43zfHQPKSwx2IJuJI5Q==,0,0,0,0,0,0
1,Vwf7mI0tTGC4amYJPD6uJg==,0,0,0,0,0,0
2,TRudRczSQt6dwb6EeZ1RLA==,0,0,0,0,0,0
3,VjG87+cBSB2B1+loMmoHCg==,0,0,0,0,0,0
4,7CfP6Hq5Qy6J0rXIuTc7kw==,0,0,0,0,0,0


In [138]:
df2_res=fill_df2_res(df2_res, df2_subset)

In [139]:
df2_res.head()

,member_id,icd_I10,icd_I10_chronic,icd_I739,icd_I739_chronic,icd_neither,icd_neither_chronic
0,nr+43zfHQPKSwx2IJuJI5Q==,1,0,0,0,0,0
1,Vwf7mI0tTGC4amYJPD6uJg==,0,0,1,1,0,0
2,TRudRczSQt6dwb6EeZ1RLA==,0,0,1,1,0,0
3,VjG87+cBSB2B1+loMmoHCg==,0,0,1,1,0,0
4,7CfP6Hq5Qy6J0rXIuTc7kw==,1,0,0,0,0,0


In [140]:
df2_res.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2841 entries, 0 to 2840
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   member_id            2841 non-null   object
 1   icd_I10              2841 non-null   int64 
 2   icd_I10_chronic      2841 non-null   int64 
 3   icd_I739             2841 non-null   int64 
 4   icd_I739_chronic     2841 non-null   int64 
 5   icd_neither          2841 non-null   int64 
 6   icd_neither_chronic  2841 non-null   int64 
dtypes: int64(6), object(1)
memory usage: 155.5+ KB


In [141]:
df2_res['icd_neither'].unique()

array([0, 1])

In [142]:
# save the data to a new csv file
datapath = '../Outputs'
save_file(df2_res, 'df2_res.csv', datapath)

A file already exists with this name.

Do you want to overwrite? (Y/N)Y
Writing file.  "../Outputs/df2_res.csv"


In [143]:
df2_res.nunique() # check df2_res

member_id              2841
icd_I10                   2
icd_I10_chronic           1
icd_I739                  2
icd_I739_chronic          2
icd_neither               2
icd_neither_chronic       2
dtype: int64

In [144]:
df2_res.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2841 entries, 0 to 2840
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   member_id            2841 non-null   object
 1   icd_I10              2841 non-null   int64 
 2   icd_I10_chronic      2841 non-null   int64 
 3   icd_I739             2841 non-null   int64 
 4   icd_I739_chronic     2841 non-null   int64 
 5   icd_neither          2841 non-null   int64 
 6   icd_neither_chronic  2841 non-null   int64 
dtypes: int64(6), object(1)
memory usage: 155.5+ KB


Notice that `icd_I10_chronic` has only one value, which is zero. We check the dataframe 'df2_subset' to see whether this is correct. What we find below shows that our function 'fill_df2_res()' works fine.

In [145]:
df2_subset[df2_subset['icd_code']=='I10']['icd_chronic_or_acute'].value_counts()

Unknown    1232
Name: icd_chronic_or_acute, dtype: int64

### 2.9.3 Labs Data Converted<a id='2.9.3_Labs_Data'></a>

In [146]:
df3_subset.head()

,member_id,loinc_code,abnormal_code_updated
531,OEcf/fmaTq+aPoYfknZK3A==,4548-4,H
532,OEcf/fmaTq+aPoYfknZK3A==,2161-8,NORM
533,OEcf/fmaTq+aPoYfknZK3A==,742-7,NORM
534,OEcf/fmaTq+aPoYfknZK3A==,777-3,H
535,OEcf/fmaTq+aPoYfknZK3A==,2028-9,NORM


In [147]:
# save the data to a new csv file
datapath = '../Outputs'
save_file(df3_subset, 'df3_subset.csv', datapath)

A file already exists with this name.

Do you want to overwrite? (Y/N)Y
Writing file.  "../Outputs/df3_subset.csv"


In [149]:
df3_res=df3_one_record_per_member_id(df3_subset)

In [150]:
df3_res.head()

,member_id,lab_4548-4,lab_4548-4_res,lab_2161-8,lab_2161-8_res,lab_742-7,lab_742-7_res,lab_777-3,lab_777-3_res,lab_2028-9,...,lab_6821-3,lab_6821-3_res,lab_6152-3,lab_6152-3_res,lab_47365-2,lab_47365-2_res,lab_32769-2,lab_32769-2_res,lab_74678-4,lab_74678-4_res
0,OEcf/fmaTq+aPoYfknZK3A==,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,OHAaSbjjQquqw750i32wuA==,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,OIW4s4BKSo+YALd/7/FNlw==,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,OKbldt8uTDOJhCtD5OrD7A==,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,OLXnSA0JSzGQhBaRRowhQw==,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [150]:
#df3_res=fill_df3_res(df3_res, df3_subset)

In [154]:
df3_res=pd.read_csv('../Outputs/df3_res.csv')
df3_res.head()

,member_id,lab_4548-4,lab_4548-4_res,lab_2161-8,lab_2161-8_res,lab_742-7,lab_742-7_res,lab_777-3,lab_777-3_res,lab_2028-9,...,lab_6821-3,lab_6821-3_res,lab_6152-3,lab_6152-3_res,lab_47365-2,lab_47365-2_res,lab_32769-2,lab_32769-2_res,lab_74678-4,lab_74678-4_res
0,OEcf/fmaTq+aPoYfknZK3A==,1,3.0,1,2.0,1,2.0,1,3.0,1,...,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0
1,OHAaSbjjQquqw750i32wuA==,1,2.0,0,0.0,1,2.0,1,2.0,1,...,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0
2,OIW4s4BKSo+YALd/7/FNlw==,1,3.0,1,2.0,1,2.0,1,2.0,1,...,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0
3,OKbldt8uTDOJhCtD5OrD7A==,1,3.0,0,0.0,1,2.0,1,2.0,1,...,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0
4,OLXnSA0JSzGQhBaRRowhQw==,1,2.0,0,0.0,1,2.0,1,2.0,1,...,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0


In [155]:
df3_res.replace({'L': 1, 'NORM':2, 'H': 3, 'ABNORM':4 }, inplace=True)

In [156]:
df3_res.head()

,member_id,lab_4548-4,lab_4548-4_res,lab_2161-8,lab_2161-8_res,lab_742-7,lab_742-7_res,lab_777-3,lab_777-3_res,lab_2028-9,...,lab_6821-3,lab_6821-3_res,lab_6152-3,lab_6152-3_res,lab_47365-2,lab_47365-2_res,lab_32769-2,lab_32769-2_res,lab_74678-4,lab_74678-4_res
0,OEcf/fmaTq+aPoYfknZK3A==,1,3.0,1,2.0,1,2.0,1,3.0,1,...,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0
1,OHAaSbjjQquqw750i32wuA==,1,2.0,0,0.0,1,2.0,1,2.0,1,...,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0
2,OIW4s4BKSo+YALd/7/FNlw==,1,3.0,1,2.0,1,2.0,1,2.0,1,...,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0
3,OKbldt8uTDOJhCtD5OrD7A==,1,3.0,0,0.0,1,2.0,1,2.0,1,...,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0
4,OLXnSA0JSzGQhBaRRowhQw==,1,2.0,0,0.0,1,2.0,1,2.0,1,...,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0


In [157]:
df3_res.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2290 entries, 0 to 2289
Columns: 3319 entries, member_id to lab_74678-4_res
dtypes: float64(1457), int64(1861), object(1)
memory usage: 58.0+ MB


Next, we will change the object dtypes to 'float64' for all columns except for `member_id`.

In [158]:
lst_object_type=list(df3_res.select_dtypes(include=['object']).columns[1:])
lst_object_type
dict0={}
for item in lst_object_type:
    dict0[item]='float64'
dict0
df3_res.astype(dict0)

,member_id,lab_4548-4,lab_4548-4_res,lab_2161-8,lab_2161-8_res,lab_742-7,lab_742-7_res,lab_777-3,lab_777-3_res,lab_2028-9,...,lab_6821-3,lab_6821-3_res,lab_6152-3,lab_6152-3_res,lab_47365-2,lab_47365-2_res,lab_32769-2,lab_32769-2_res,lab_74678-4,lab_74678-4_res
0,OEcf/fmaTq+aPoYfknZK3A==,1,3.0,1,2.0,1,2.0,1,3.0,1,...,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0
1,OHAaSbjjQquqw750i32wuA==,1,2.0,0,0.0,1,2.0,1,2.0,1,...,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0
2,OIW4s4BKSo+YALd/7/FNlw==,1,3.0,1,2.0,1,2.0,1,2.0,1,...,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0
3,OKbldt8uTDOJhCtD5OrD7A==,1,3.0,0,0.0,1,2.0,1,2.0,1,...,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0
4,OLXnSA0JSzGQhBaRRowhQw==,1,2.0,0,0.0,1,2.0,1,2.0,1,...,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2285,/2Glid0uTfiICiJpFqoRQA==,0,0.0,0,0.0,1,2.0,1,2.0,1,...,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0
2286,/6ErI2QeS9+NYAvCfq4NEg==,0,0.0,0,0.0,0,0.0,1,2.0,1,...,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0
2287,/6KVmD1fSLyLfmsKwAkHYA==,0,0.0,0,0.0,0,0.0,0,0.0,1,...,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0
2288,//CCSUI/S9ucTP+bePc5bg==,1,3.0,1,NaN,1,2.0,1,2.0,1,...,0,0.0,0,0.0,0,0.0,0,0.0,1,NaN


In [159]:
df3_res.info() # check the update

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2290 entries, 0 to 2289
Columns: 3319 entries, member_id to lab_74678-4_res
dtypes: float64(1457), int64(1861), object(1)
memory usage: 58.0+ MB


In [160]:
# save the data to a new csv file
datapath = '../Outputs'
save_file(df3_res, 'df3_res.csv', datapath)

A file already exists with this name.

Do you want to overwrite? (Y/N)Y
Writing file.  "../Outputs/df3_res.csv"


### 2.9.4 Patients Data Converted<a id='2.9.4_Patients_Data'></a>

In [161]:
df4_subset

,member_id,age,sex,dual_status,insurance_company,insurance_type,pbp_type,county,city
0,AAHLhKW7RYOEQIemWrh/1w==,72,M,1,1,1,1,1,1
23,ADD7uXD9TBOUjJD43H3tNw==,46,F,2,2,1,3,2,2
56,AH8v69t8QBW1u2Iuh/5qlQ==,43,M,2,5,1,1,9,20
58,AIEq/y06TGOCCjwkDzmaeA==,89,M,2,2,1,2,15,35
60,AIfxyYzERH6jOgV6KZqn+g==,76,F,2,2,1,2,2,2
...,...,...,...,...,...,...,...,...,...
26503,/2Glid0uTfiICiJpFqoRQA==,79,F,2,2,1,5,2,82
26519,/6ErI2QeS9+NYAvCfq4NEg==,87,M,2,2,1,6,8,28
26520,/6KVmD1fSLyLfmsKwAkHYA==,71,M,6,1,1,3,18,78
26553,//CCSUI/S9ucTP+bePc5bg==,77,F,2,4,1,1,7,10


In [162]:
df4_subset.nunique()

member_id            2444
age                    58
sex                     2
dual_status             8
insurance_company      11
insurance_type          2
pbp_type                9
county                 30
city                  202
dtype: int64

'df4_subset' has one record per member id, which indicates that we only need to get dummy variables for the categorical columns.

In [163]:
df4_subset=df4_subset.astype({'age': 'int64', 'sex': 'category', 'dual_status':'category', 'insurance_company':'category',
       'insurance_type': 'category', 'pbp_type': 'category', 'county': 'category', 'city': 'category'})

In [164]:
df4_subset.info() # check the update

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2444 entries, 0 to 26558
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype   
---  ------             --------------  -----   
 0   member_id          2444 non-null   object  
 1   age                2444 non-null   int64   
 2   sex                2444 non-null   category
 3   dual_status        2444 non-null   category
 4   insurance_company  2444 non-null   category
 5   insurance_type     2444 non-null   category
 6   pbp_type           2444 non-null   category
 7   county             2444 non-null   category
 8   city               2444 non-null   category
dtypes: category(7), int64(1), object(1)
memory usage: 90.8+ KB


In [165]:
# save the data to a new csv file
datapath = '../Outputs'
save_file(df4_subset, 'df4_subset.csv', datapath)

A file already exists with this name.

Do you want to overwrite? (Y/N)Y
Writing file.  "../Outputs/df4_subset.csv"


In [166]:
df4_res=pd.get_dummies(df4_subset, columns=['sex', 'dual_status', 'insurance_company',
       'insurance_type', 'pbp_type', 'county', 'city'])

In [167]:
df4_res.head()

,member_id,age,sex_F,sex_M,dual_status_1,dual_status_2,dual_status_3,dual_status_4,dual_status_5,dual_status_6,...,city_9,city_90,city_91,city_92,city_93,city_95,city_96,city_97,city_98,city_99
0,AAHLhKW7RYOEQIemWrh/1w==,72,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
23,ADD7uXD9TBOUjJD43H3tNw==,46,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
56,AH8v69t8QBW1u2Iuh/5qlQ==,43,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
58,AIEq/y06TGOCCjwkDzmaeA==,89,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
60,AIfxyYzERH6jOgV6KZqn+g==,76,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [168]:
# save the data to a new csv file
datapath = '../Outputs'
save_file(df4_res, 'df4_res.csv', datapath)

A file already exists with this name.

Do you want to overwrite? (Y/N)Y
Writing file.  "../Outputs/df4_res.csv"


### 2.9.5  Prescriptions Data Converted<a id='2.9.5_Prescriptions_Data'></a>

In [169]:
df5_subset.head()

,member_id,ndc_number,new_or_refill,drug_quantity_rate
0,AAHLhKW7RYOEQIemWrh/1w==,60505013400,N,1.0
1,AAHLhKW7RYOEQIemWrh/1w==,62175013643,N,1.0
2,AAHLhKW7RYOEQIemWrh/1w==,50742013510,N,1.0
3,AAHLhKW7RYOEQIemWrh/1w==,60505267108,N,1.0
4,AAHLhKW7RYOEQIemWrh/1w==,43547039910,N,1.0


In [170]:
# save the data to a new csv file
datapath = '../Outputs'
save_file(df5_subset, 'df5_subset.csv', datapath)

A file already exists with this name.

Do you want to overwrite? (Y/N)Y
Writing file.  "../Outputs/df5_subset.csv"


In [168]:
df5_res=df5_one_record_per_member_id(df5_subset)

In [169]:
df5_res.head()

,member_id,ndc#_60505013400,ndc#_60505013400_rate,ncd#_60505013400_refill,ndc#_62175013643,ndc#_62175013643_rate,ncd#_62175013643_refill,ndc#_50742013510,ndc#_50742013510_rate,ncd#_50742013510_refill,...,ncd#_93739286_refill,ndc#_40093010390,ndc#_40093010390_rate,ncd#_40093010390_refill,ndc#_50924047550,ndc#_50924047550_rate,ncd#_50924047550_refill,ndc#_66758019005,ndc#_66758019005_rate,ncd#_66758019005_refill
0,AAHLhKW7RYOEQIemWrh/1w==,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ADD7uXD9TBOUjJD43H3tNw==,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,AH8v69t8QBW1u2Iuh/5qlQ==,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,AIEq/y06TGOCCjwkDzmaeA==,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,AIfxyYzERH6jOgV6KZqn+g==,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [170]:
#df5_res=fill_df5_res(df5_res, df5_subset)

In [171]:
# save the data to a new csv file
datapath = '../Outputs'
save_file(df5_res, 'df5_res.csv', datapath)

Writing file.  "../Outputs/df5_res.csv"


In [172]:
df5_res.nunique()

member_id                  2410
ndc#_60505013400              2
ndc#_60505013400_rate         3
ncd#_60505013400_refill       1
ndc#_62175013643              2
                           ... 
ndc#_10135018210_refill       1
ndc#_832102510_refill         1
ndc#_71093012105_refill       1
ndc#_43598075360_refill       1
ndc#_64380086106_refill       1
Length: 26503, dtype: int64

In [177]:
df5.head()

,member_id,ndc_number,new_or_refill,drug_quantity_rate
0,AAHLhKW7RYOEQIemWrh/1w==,60505013400,N,1.0
1,AAHLhKW7RYOEQIemWrh/1w==,62175013643,N,1.0
2,AAHLhKW7RYOEQIemWrh/1w==,50742013510,N,1.0
3,AAHLhKW7RYOEQIemWrh/1w==,60505267108,N,1.0
4,AAHLhKW7RYOEQIemWrh/1w==,43547039910,N,1.0


In [180]:
df5[(df5['member_id']=='AH8v69t8QBW1u2Iuh/5qlQ==')& (df5['ndc_number']=='52565001480') ]

,member_id,ndc_number,new_or_refill,drug_quantity_rate
3180,AH8v69t8QBW1u2Iuh/5qlQ==,52565001480,R,0.375
3192,AH8v69t8QBW1u2Iuh/5qlQ==,52565001480,R,0.375
3202,AH8v69t8QBW1u2Iuh/5qlQ==,52565001480,R,0.375


In [173]:
df5_res.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2410 entries, 0 to 2409
Columns: 26503 entries, member_id to ndc#_64380086106_refill
dtypes: float64(9220), int64(17282), object(1)
memory usage: 487.3+ MB


In [174]:
df5_res[df5_res['member_id']=='AH8v69t8QBW1u2Iuh/5qlQ==']['ndc#_52565001480_rate']
# check df5_res

2    0.375
Name: ndc#_52565001480_rate, dtype: float64

In [175]:
df5_res['ndc#_591034301_refill'].unique()

array([nan,  1.])

In [171]:
df5_res=pd.read_csv('../Outputs/df5_res.csv')
df5_res.head()

,member_id,ndc#_60505013400,ndc#_60505013400_rate,ncd#_60505013400_refill,ndc#_62175013643,ndc#_62175013643_rate,ncd#_62175013643_refill,ndc#_50742013510,ndc#_50742013510_rate,ncd#_50742013510_refill,...,ndc#_228300311_refill,ndc#_172392760_refill,ndc#_59746038410_refill,ndc#_68382013710_refill,ndc#_16252060102_refill,ndc#_10135018210_refill,ndc#_832102510_refill,ndc#_71093012105_refill,ndc#_43598075360_refill,ndc#_64380086106_refill
0,AAHLhKW7RYOEQIemWrh/1w==,1,1.0,0,1,1.0,0,1,1.0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ADD7uXD9TBOUjJD43H3tNw==,0,0.0,0,0,0.0,0,0,0.0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AH8v69t8QBW1u2Iuh/5qlQ==,0,0.0,0,0,0.0,0,0,0.0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AIEq/y06TGOCCjwkDzmaeA==,0,0.0,0,0,0.0,0,0,0.0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AIfxyYzERH6jOgV6KZqn+g==,0,0.0,0,0,0.0,0,0,0.0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


We have nulls in our 'df5_res' dataframe, which are in fact supposed to be 0. We will replace nulls with zero. 

### Part C <a id='Part_C'></a>    

### 2.10  Join the Converted Dataframes<a id='2.10_Join_the_Dataframes'></a>

Here we will join the dataframes 'df1_res', 'df2_res', 'df3_res', 'df4_res', 'df5_res' to get the dataframe we will use in our model.

In [172]:
lst=[df1_res, df3_res, df4_res, df5_res]
df_final=df2_res
for df_ in lst:
    df_final=df_final.merge(df_, how='left', on='member_id')
df_final   

,member_id,icd_I10,icd_I10_chronic,icd_I739,icd_I739_chronic,icd_neither,icd_neither_chronic,num_of_visit,v1,v1_re_adm,...,ndc#_228300311_refill,ndc#_172392760_refill,ndc#_59746038410_refill,ndc#_68382013710_refill,ndc#_16252060102_refill,ndc#_10135018210_refill,ndc#_832102510_refill,ndc#_71093012105_refill,ndc#_43598075360_refill,ndc#_64380086106_refill
0,nr+43zfHQPKSwx2IJuJI5Q==,1,0,0,0,0,0,1.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Vwf7mI0tTGC4amYJPD6uJg==,0,0,1,1,0,0,1.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,TRudRczSQt6dwb6EeZ1RLA==,0,0,1,1,0,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,VjG87+cBSB2B1+loMmoHCg==,0,0,1,1,0,0,2.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,7CfP6Hq5Qy6J0rXIuTc7kw==,1,0,0,0,0,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2836,rlUlZUMgRTim7Mkdf1yaPg==,0,0,0,0,1,1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2837,KY3kRZ3uQcqW6+7kl4PbWw==,0,0,0,0,1,1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2838,71rJt0vGQTaGp1yQ9jBB/g==,0,0,0,0,1,0,4.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2839,lzOoZy24Qc2CxCC1i1wsEA==,0,0,0,0,1,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### 2.11  Save the Final Dataframe<a id='2.11_Save_the_Data'></a>

In [173]:
# save the data to a new csv file
datapath = '../Outputs'
save_file(df_final, 'df_final.csv', datapath)

Writing file.  "../Outputs/df_final.csv"


### 2.12  Final Remarks<a id='2.12_Final_Remarks'></a>

We have answered the following questions:

* Q1) What are the required target illnesses?
* A1) We classify our patients using data from 2018-2019. Patients are classified in the following ways: patients who will have one, both, or neither of diseases 'I10' and 'I739'.


* Q2) Are there potentially useful features?
* A2) We have 25347 features that are potentially useful. We will use feature selection algorithms to pick the most useful of these features.


* Q3) Are there any fundamental issues with the data?
* A3) We had numerous issues with the data. Most are fixed. The final dataframe is sparse. We will consider this fact when we choose our model. 
   